In [1]:
class RESTObject(PipeProcessingNode):    
    def pipe_construct(self):
        self.addPipe(name='parameters', type=GeneralConfigure, io='input', examples=[{},])
        self.addPipe(name='get', type=RESTObject.get, io='input', examples=[{},{'property':5},{'mongo_query':'mongo_style_query'}])
        self.addPipe(name='output', type=GeneralOutput, io='output', dependency='get',examples=[{'id':10,'p2':'value2'},])
        
    def get(feature,context= None):
        ## Easy -- find yourself
        # expect, feture to be a single id. Return some json
        ret = flex()
        #ret.name ='name'
        #ret.height ='tall'
        if feature:
            ret= feature
        return feature*10

#### Lets understand how a ProcessingNetwork uses a node:
#
# 1. Run with some hardcoded values to test
#
r = RESTObject(settings={'name':'travis'},
               dependency_list={'get':5})
print(r.process({}))

#
# 2. Run on some data that is in the array
#
r = RESTObject(settings={'name':'travis'},
               dependency_list={'get':('input',)})
print(r.process({'input':2}))


#
# 3. Run on some data buried deep in an array
#
r = RESTObject(settings={'name':'travis'},
               dependency_list={'get':('input','x_values','height',)})
print(r.process({'input':{'x_values':{'height':10}}}))

#
# 4. Flex example. Use flex to clean up dictionay readability
#
p = flex()
p.settings.name='travis'
p.dependencies.get = ('input','x_values','height',)
r = RESTObject(settings=p.settings, dependency_list=p.dependencies)

data = flex()
data.input.x_values.height = 10
print (r.process(data))

#
# 5. But... a processing network is an object that registers and manages these dependencies. In fact, it does graph analysis, 
# and all kinds of optimizations so that developers can dynamically include, swap, and play with dependencues, all in a 
# format can that be snapshotted with a dictionary.

ndef = flex()
data = flex()

ndef.travis.type=RESTObject
ndef.travis.dependencies.get=('input','x_values','height',)

pn=ProcessingNetwork(ndef)
data.input.x_values.height = 10
print(pn.process(data))


#
# 6. Ok, this does not seem like it saves effort, until you start to chain objects
# With two lines of code, we add a derivitive filter with a dependency on travis
#

ndef = flex()
data = flex()

ndef.travis.type=RESTObject
ndef.travis.dependencies.get=('input','x_values','height',)
ndef.brian.type=RESTObject
ndef.brian.dependencies.get=('travis','output',)

pn=ProcessingNetwork(ndef)
data.input.x_values.height = 10
print(pn.process(data))



NameError: name 'PipeProcessingNode' is not defined